***
*Project:* Deep Active Inference

*Author:* Jingwei Liu (Music department, UC San Diego)
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Tutorial</span> 


# Version 8: Synchronization (hidden state distributions; one-side adaption)

* Real-time synchronization

* Access to hidden state distributions

* One-side adaption (agent)

* Minimize under KL-Divergence

In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [3]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:18000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[18000:,:])

In [5]:
batch_size = 18

In [6]:
n_steps = 20
window_length = n_steps
dataset = test_dataset.window(window_length, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)

In [7]:
def creat_model(batch_size,Type):
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(None,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
    reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)
    if Type == "_0_1":
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)
    elif Type == "_1_0":
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

    aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
    output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
    output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1, output_B2])
    
    aux_model1 = keras.models.Model([input_A, input_B], output_B1)
    aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
    
    sub_model1 = keras.models.Model(input_A, output_A)
    sub_model2 = keras.models.Model(input_A, output_B2)
    
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    return model,aux_model1,aux_model2,sub_model1,sub_model2,att_model

In [8]:
model_0_1_,aux_model1_0_1_,aux_model2_0_1_, sub_model1_0_1_,sub_model2_0_1_,att_model_0_1_ = creat_model(batch_size,'_0_1')
model_1_0_,aux_model1_1_0_,aux_model2_1_0_, sub_model1_1_0_,sub_model2_1_0_,att_model_1_0_ = creat_model(batch_size,'_1_0')

In [9]:
model_1_0_.load_weights('model_1_0_weight.h5')

In [32]:
model_0_1_.load_weights('version8_model_0_1_step1_train1.h5')

In [35]:
model_0_1_.load_weights('version8_model_0_1_step2_train1.h5')

In [38]:
model_0_1_.load_weights('version8_model_0_1_step5_train1.h5')

In [110]:
model_0_1_.load_weights('version8_model_0_1_step10_train1.h5')

In [96]:
model_0_1_.load_weights('version8_model_0_1_step1_train1_random.h5')

In [99]:
model_0_1_.load_weights('version8_model_0_1_step2_train1_random.h5')

In [127]:
model_0_1_.load_weights('version8_model_0_1_step5_train1_random.h5')

In [128]:
# version 4/5/6/7/8/9/10 generate sequence

seq_length = 10

#generate_mode = "max"
generate_mode = "random"

for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    pred_0_1_1 = np.zeros((batch_size,seq_length))
    pred_1_0_0 = np.zeros((batch_size,seq_length))
    
    obv_weight = np.zeros((batch_size,seq_length))
    act_weight = np.zeros((batch_size,seq_length))

    for s in range(seq_length):

        [pred_o,a] = aux_model2_0_1_.predict(initial) #(18,30)

        for i in range(batch_size):
            if generate_mode == "max":
                pred_0_1_1[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif generate_mode == "random":
                pred_0_1_1[i,s] = np.random.choice(30, 1, p=a[i,:])[0]


        [pred_a,o] = aux_model2_1_0_.predict(initial) #(18,30)
        
        for i in range(batch_size):
            if generate_mode == "max":
                pred_1_0_0[i,s] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
            elif generate_mode == "random":
                pred_1_0_0[i,s] = np.random.choice(30, 1, p=o[i,:])[0]
            
            index = int(pred_1_0_0[i,s])
            obv_weight[i,s] = pred_o[i,index]
            
            index = int(pred_0_1_1[i,s])
            act_weight[i,s] = pred_a[i,index]
            
        
        one_hot = tf.one_hot(pred_0_1_1[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_0_1_1 = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
        
        one_hot = tf.one_hot(pred_1_0_0[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_1_0_0 = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)

        sequence = tf.concat((sequence,out_1_0_0,out_0_1_1),1)
        initial = sequence[:,-20:,:]
        
    print('action:',pred_0_1_1)
    print('observation:',pred_1_0_0)
    print('')
    
    
    step = [];
    for j in range(seq_length):
        step.append("step" + str(j))
    step.append("sequence match")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence" + str(i))
    sample.append("step match")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(pred_0_1_1[i,j])) + "/(" + '%.2f'%(act_weight[i,j]) + ")")
        match_a = np.sum(act_weight[i,:])/seq_length
        seq.append('%.2f'%(match_a))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match_a = np.sum(act_weight[:,j])/batch_size
        last_row.append('%.2f'%(match_a))
    total_a = np.sum(act_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total_a))
    table.append(last_row)
    
    format_row = "{:>11}" * (seq_length + 2)
    print(format_row.format("act/(pred_act_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))

    print('')
        
    table2 = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(pred_1_0_0[i,j])) + "/(" + '%.2f'%(obv_weight[i,j]) + ")")
        match_o = np.sum(obv_weight[i,:])/seq_length
        seq.append('%.2f'%(match_o))
        table2.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match_o = np.sum(obv_weight[:,j])/batch_size
        last_row.append('%.2f'%(match_o))
    total_o = np.sum(obv_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total_o))
    table2.append(last_row)
    
    format_row = "{:>11}" * (seq_length + 2)
    print(format_row.format("obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table2):
        print(format_row.format(team, *row))

action: [[18. 16. 15. 23. 20. 23. 20. 17. 21. 17.]
 [20. 18. 24. 21. 17. 17. 23. 20. 23. 23.]
 [17. 19. 16. 16. 12. 20. 19. 15. 18. 25.]
 [21. 17. 23. 17. 15. 18. 10. 15.  9.  6.]
 [15. 16. 15. 14.  7. 22. 10. 14. 13. 17.]
 [18. 15. 16. 16. 10. 13. 20. 15. 16. 21.]
 [20. 17. 19. 19. 25. 17. 14. 15. 11. 14.]
 [18. 13. 17. 24. 16. 20. 19. 19. 17.  7.]
 [19. 21. 19. 19. 20. 15. 20. 15. 11.  9.]
 [ 6. 12. 14. 11. 13.  8. 12. 12. 11.  9.]
 [14. 15. 19. 16. 24. 21. 16. 20. 22. 19.]
 [18. 20. 20. 13. 11. 11. 21. 13. 20. 17.]
 [12. 17. 16. 23. 26. 18. 11. 14. 15. 16.]
 [16. 21. 16. 16. 14. 14. 11. 11. 13. 14.]
 [ 8.  5.  2.  3.  5. 14.  7.  9. 12. 10.]
 [17. 24. 27. 15. 16. 19. 18. 15. 16. 19.]
 [19. 13. 10.  9. 14. 10. 14.  5.  8.  8.]
 [15.  9.  7. 10.  6. 14. 15. 11. 16.  7.]]
observation: [[21. 29. 28. 26. 26. 25. 29. 26. 24. 20.]
 [29. 28. 28. 26. 29. 29. 23. 27. 25. 28.]
 [22. 22. 22. 27. 26. 25. 24. 29. 27. 25.]
 [29. 27. 22. 25. 25. 18. 21. 21. 18. 18.]
 [18. 21. 21. 22. 20. 17. 18. 16

In [129]:
for sample in range(batch_size):
    music = tf.reshape(sequence[sample,:,:],[-1,31])
    #music = music[-20:,:]
    row,col = music.shape
    cantus_lst = [];
    counter_lst = [];
    for i in range(row):
        clas = np.where(music[i,:-1] == 1)[0][0]
        if music[i,-1] == 0:
            cantus_lst.append(clas+43)
        elif music[i,-1] == 1:
            counter_lst.append(clas+55)
    print(np.array(counter_lst) - np.array(cantus_lst))

[ 8  3  3  7  9  9  4  7  8  4  9 -1 -1  9  6 10  3  3  9  9]
[ 4  7  8  8  8  9  3  4  3  4  3  2  8  7  0  0 12  5 10  7]
[ 8  9  3  4  3  4  9  9 12  7  7  9  6  1 -2  7  7 -2  3 12]
[ 4  8  3  8  9  3  7  9 12  7  4  2 13  4  2 12  1  6  3  0]
[ 3  4  4  7 12  7  3  9  7  4  9  7  6  4 -1 17  4 10 15 17]
[ 8 12  3  7  4  4  9  3  7  4  3 -2 10  8 -1  4 11 14  9 12]
[ 3  8  9  9  8  8  4  4  9  3  9  4  6  3  9  4  4  9 -4  7]
[ 7  9  7  3  9  4  9  4  7  8 10  6  8 10  6  8  8 12 12  5]
[ 4  8  8  9  3  8  9 12  3  7  6  7  2  6  7  5 14  8  4  3]
[ 4  7  9  9  8  8  9  4  3  3  9 16 15  7  6  1  4  2  3  0]
[ 7 12  8  9  3  8  8  9  7  3  6  6  8  0  9  6  3  4 11  9]
[12  9  8  9  8  9  4  7  4  7  6 10  9 -4 -3  2 11  1 11  9]
[ 3  7  8 12  3  4  9  9  7  4 -2  2  0 12 14  4 -5  6  5  8]
[ 8  3  4  8  4  4  9  7 12  9  5 15  5  7  4  4  7 10 10 15]
[ 7  3  4  4  9 12  8  8  8  4  9  9  5 10  6 10 -1  9 11  5]
[ 3  4  7  8  8  9  9  9  4  3  5 15 10  2  9  8 13 -2  0  3]
[ 3 12  

In [130]:
sample = 3
music = tf.reshape(sequence[sample,:,:],[-1,31])
#music = music[-20:,:]
row,col = music.shape
cantus_lst = [];
counter_lst = [];
for i in range(row):
    clas = np.where(music[i,:-1] == 1)[0][0]
    if music[i,-1] == 0:
        cantus_lst.append(clas+43)
    elif music[i,-1] == 1:
        counter_lst.append(clas+55)

In [131]:
cantus_lst

[58,
 62,
 66,
 64,
 65,
 70,
 69,
 70,
 68,
 69,
 72,
 70,
 65,
 68,
 68,
 61,
 64,
 64,
 61,
 61]

In [132]:
counter_lst

[62,
 70,
 69,
 72,
 74,
 73,
 76,
 79,
 80,
 76,
 76,
 72,
 78,
 72,
 70,
 73,
 65,
 70,
 64,
 61]

In [133]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([ 4,  8,  3,  8,  9,  3,  7,  9, 12,  7,  4,  2, 13,  4,  2, 12,  1,
        6,  3,  0], dtype=int64)

In [134]:
np.diff(counter_lst)

array([ 8, -1,  3,  2, -1,  3,  3,  1, -4,  0, -4,  6, -6, -2,  3, -8,  5,
       -6, -3], dtype=int64)

In [135]:
np.diff(cantus_lst)

array([ 4,  4, -2,  1,  5, -1,  1, -2,  1,  3, -2, -5,  3,  0, -7,  3,  0,
       -3,  0], dtype=int64)

In [136]:
s = Session()
s.tempo = 200
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano
Using preset Piano Merlin for piano
